# Data generation: using python to sweep over methods and parameters

In this notebook, we illustrate how to use python to generate and run a list of commands. In this example, we generate a list of QIIME 1.9.0 ``assign_taxonomy.py`` commands, though this workflow for command generation is generally very useful for performing *parameter sweeps* (i.e., exploration of sets of parameters for achieving a specific result for comparative purposes). 

## Environment preparation

In [3]:
from os import system
from os.path import join, expandvars, exists
from joblib import Parallel, delayed
from glob import glob
from tax_credit.framework_functions import (recall_novel_taxa_dirs,
                                            parameter_sweep,
                                            move_results_to_repository)


In [20]:
project_dir = '../../../tax-credit-data-bk/'
analysis_name= "cross-validated"

results_dir = join('..', '..', 'cross-validated-silva')

## Preparing data set sweep

First, we're going to define the data sets that we'll sweep over. The "cross-validated" simulated reads that we use here are subsets of reference sequence databases generated duing the novel-taxa analysis. We re-use these data sets here for the purposes of having cross-validated data subsets. As the cross-validated dataset names depend on how the database generation notebook was executed, we must define the variables used to create these datasets. If you modified any variables in that notebook, set these same variables below. If you did not, then do not modify.

recall_novel_taxa_dirs() generates a list of dataset_reference_combinations and a dictionary of reference_dbs mapped to each dataset, which we feed to parameter_sweep below.

In [13]:
iterations = 10
data_dir = join(project_dir, "data", analysis_name)
# databases is a list of names given as dictionary keys in the second
# cell of the database generation notebook. Just list the names here.
databases = ['S1-REF']

# Generate a list of input directories
(dataset_reference_combinations, reference_dbs) = recall_novel_taxa_dirs(\
    data_dir, databases, iterations, ref_seqs='ref_seqs.fasta',
    ref_taxa='ref_taxa.tsv', max_level=6, min_level=5, multilevel=False)

## Preparing the method/parameter combinations and generating commands

Now we set the methods and method-specific parameters that we want to sweep. Modify to sweep other methods. Note how method_parameters_combinations feeds method/parameter combinations to parameter_sweep() in the cell below.

### Assignment Using QIIME 1 or Command-Line Classifiers

Here we provide an example of taxonomy assignment using legacy ``QIIME 1`` classifiers executed on the command line. To accomplish this, we must first convert ``commands`` to a string, which we then pass to bash for execution. As ``QIIME 1`` is written in python-2, we must also activate a separate environment in which QIIME 1 [has been installed](http://qiime.org/install/install.html). If any environmental variables need to be set (in this example, the [RDP_JAR_PATH](http://qiime.org/install/alternative.html#rdp-jar-path-environment-variable)), we must also source the .bashrc file.

In [4]:
method_parameters_combinations = { # probabalistic classifiers
              'rdp': {'confidence': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5,
                                     0.6, 0.7, 0.8, 0.9, 1.0]},
              
              # global alignment classifiers
              'uclust': {'min_consensus_fraction': [0.51, 0.76, 1.0], 
                         'similarity': [0.8, 0.9],
                         'uclust_max_accepts': [1, 3, 5]},
             
              # local alignment classifiers
              'sortmerna': {'sortmerna_e_value': [1.0],
                            'min_consensus_fraction': [0.51, 0.76, 1.0], 
                            'similarity': [0.8, 0.9],
                            'sortmerna_best_N_alignments ': [1, 3, 5],
                            'sortmerna_coverage' : [0.8, 0.9]},
              'blast' : {'blast_e_value' : [0.0000000001, 0.001, 1, 1000]}
             }

Now enter the template of the command to sweep, and generate a list of commands with ``parameter_sweep()``.

Fields must adhere to following format:

                      {0} = output directory
                      {1} = input data
                      {2} = output destination
                      {3} = reference taxonomy
                      {4} = method name
                      {5} = other parameters

In [5]:
command_template = 'bash -c "source activate qiime1; source ./.bashrc; mkdir -p {0} ; assign_taxonomy.py -v -i {1} -o {0} -r {2} -t {3} -m {4} {5} --rdp_max_memory 96000"'
        
commands = parameter_sweep(data_dir, results_dir, reference_dbs,
                           dataset_reference_combinations,
                           method_parameters_combinations, command_template,
                           infile='query.fasta')


As a sanity check, we can look at the first command that was generated and the number of commands generated.

In [6]:
for method in method_parameters_combinations:
    print(method)
    for command in commands:
        if '/'+method+'/' in command and 'S1' in command:
            print(command)
            break

rdp
bash -c "source activate qiime1; source ./.bashrc; mkdir -p ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/rdp/0.0 ; assign_taxonomy.py -v -i ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/query.fasta -o ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/rdp/0.0 -r ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/ref_seqs.fasta -t ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/ref_taxa.tsv -m rdp --confidence 0.0 --rdp_max_memory 96000"
sortmerna
bash -c "source activate qiime1; source ./.bashrc; mkdir -p ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/sortmerna/0.51:0.8:1:0.8:1.0 ; assign_taxonomy.py -v -i ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/query.fasta -o ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/sortmerna/0.51:0.8:1:0.8:1.0 -r ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/ref_seqs.fasta -t ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/ref_taxa.tsv -m s

In [7]:
print(len(commands))

690


In [14]:
unrun_commands = []
for command in commands:
    cmd = command.split()
    for flag, arg in zip(cmd, cmd[1:]):
        if flag == '-o' and not exists(join(arg, 'query_tax_assignments.txt')):
            unrun_commands.append(command)

In [15]:
print(len(unrun_commands))

35


In [17]:
for command in unrun_commands:
    print(command + '\n')

bash -c "source activate qiime1; source ./.bashrc; mkdir -p ../../cross-validated-tmp/S1-REF-iter2/S1-REF-iter2/rdp/0.4 ; assign_taxonomy.py -v -i ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter2/query.fasta -o ../../cross-validated-tmp/S1-REF-iter2/S1-REF-iter2/rdp/0.4 -r ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter2/ref_seqs.fasta -t ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter2/ref_taxa.tsv -m rdp --confidence 0.4 --rdp_max_memory 96000"

bash -c "source activate qiime1; source ./.bashrc; mkdir -p ../../cross-validated-tmp/S1-REF-iter7/S1-REF-iter7/rdp/0.0 ; assign_taxonomy.py -v -i ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter7/query.fasta -o ../../cross-validated-tmp/S1-REF-iter7/S1-REF-iter7/rdp/0.0 -r ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter7/ref_seqs.fasta -t ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter7/ref_taxa.tsv -m rdp --confidence 0.0 --rdp_max_memory 96000"

bash -c "source ac

In [11]:
Parallel(n_jobs=5)(delayed(system)(command) for command in unrun_commands);

Finally, we run our commands.

In [ ]:
Parallel(n_jobs=23)(delayed(system)(command) for command in commands);

### BLAST+

In [9]:
method_parameters_combinations = {
              'blast+' : {'p-evalue': [0.001],
                          'p-maxaccepts': [1, 10],
                          'p-perc-identity': [0.80, 0.97, 0.99],
                          'p-min-consensus': [0.51, 0.75, 0.99]}
             }

command_template = ("mkdir -p {0}; "
                    "qiime feature-classifier classify-consensus-blast --i-query {1} --o-classification "
                    "{0}/rep_seqs_tax_assignments.qza --i-reference-reads {2} --i-reference-taxonomy {3} {5}; "
                    "qiime tools export {0}/rep_seqs_tax_assignments.qza --output-dir {0}; "
                    "mv {0}/taxonomy.tsv {0}/query_tax_assignments.txt")
        
(dataset_reference_combinations, reference_dbs) = recall_novel_taxa_dirs(\
    data_dir, databases, iterations, ref_seqs='ref_seqs.qza',
    ref_taxa='ref_taxa.qza', max_level=6, min_level=5, multilevel=False)

commands = parameter_sweep(data_dir, results_dir, reference_dbs,
                           dataset_reference_combinations,
                           method_parameters_combinations, command_template,
                           infile='query.qza', output_name='rep_seqs_tax_assignments.qza')

In [10]:
print(len(commands))
print(commands[0])

180
mkdir -p ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/blast+/0.001:1:0.51:0.8; qiime feature-classifier classify-consensus-blast --i-query ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/query.qza --o-classification ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/blast+/0.001:1:0.51:0.8/rep_seqs_tax_assignments.qza --i-reference-reads ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/ref_seqs.qza --i-reference-taxonomy ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/ref_taxa.qza --p-perc-identity 0.8 --p-min-consensus 0.51 --p-evalue 0.001 --p-maxaccepts 1; qiime tools export ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/blast+/0.001:1:0.51:0.8/rep_seqs_tax_assignments.qza --output-dir ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/blast+/0.001:1:0.51:0.8; mv ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/blast+/0.001:1:0.51:0.8/taxonomy.tsv ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/blast+/0.00

In [ ]:
Parallel(n_jobs=23)(delayed(system)(command) for command in commands);

### VSEARCH

In [12]:
method_parameters_combinations = {
              'vsearch' : {'p-maxaccepts': [1, 10],
                           'p-perc-identity': [0.80, 0.90, 0.97, 0.99],
                           'p-min-consensus': [0.51, 0.99]}
             }

command_template = ("mkdir -p {0}; "
                    "qiime feature-classifier classify-consensus-vsearch --i-query {1} --o-classification "
                    "{0}/rep_seqs_tax_assignments.qza --i-reference-reads {2} --i-reference-taxonomy {3} {5}; "
                    "qiime tools export {0}/rep_seqs_tax_assignments.qza --output-dir {0}; "
                    "mv {0}/taxonomy.tsv {0}/query_tax_assignments.txt")
        
commands = parameter_sweep(data_dir, results_dir, reference_dbs,
                           dataset_reference_combinations,
                           method_parameters_combinations, command_template,
                           infile='query.qza', output_name='rep_seqs_tax_assignments.qza')

In [13]:
print(len(commands))
print(commands[0])

160
mkdir -p ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/vsearch/1:0.51:0.8; qiime feature-classifier classify-consensus-vsearch --i-query ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/query.qza --o-classification ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/vsearch/1:0.51:0.8/rep_seqs_tax_assignments.qza --i-reference-reads ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/ref_seqs.qza --i-reference-taxonomy ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/ref_taxa.qza --p-perc-identity 0.8 --p-min-consensus 0.51 --p-maxaccepts 1; qiime tools export ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/vsearch/1:0.51:0.8/rep_seqs_tax_assignments.qza --output-dir ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/vsearch/1:0.51:0.8; mv ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/vsearch/1:0.51:0.8/taxonomy.tsv ../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/vsearch/1:0.51:0.8/query_tax_assignments.txt


In [ ]:
Parallel(n_jobs=23)(delayed(system)(command) for command in commands);

### scikit-learn

In [15]:
p_feat_ext__ngram_range = [
    '[6,6]', '[8,8]', '[12,12]', '[16,16]'
]

# ['[4,4]', '[6,6]', '[8,8]', '[16,16]', '[32,32]',
# '[7,7]', '[9,9]', '[10,10]', '[11,11]', 
# '[12,12]', '[14,14]', '[18,18]'],

method_parameters_combinations = {
    'naive-bayes' : {'p-feat-ext--ngram-range': p_feat_ext__ngram_range,
                     'p-classify--alpha': [0.001]},
    'naive-bayes-bespoke' : {'p-feat-ext--ngram-range': p_feat_ext__ngram_range,
                             'p-classify--alpha': [0.001],
                             'p-classify--fit-prior': ['']}
}

command_template = ('mkdir -p "{0}"; '
                    'qiime feature-classifier fit-classifier-naive-bayes --o-classifier '
                    '"{0}/classifier.qza" --i-reference-reads {2} --i-reference-taxonomy {3} {5}; ')

confidences = [0.5, 0.7, 0.9, 0.96, 0.99]

# [0.0, 0.5, 0.7, 0.9, 0.92, 0.94,
#  0.96, 0.98, 0.99]

command_template += ''.join(
    'mkdir -p "{0}:' + str(c) + '"; '
    'qiime feature-classifier classify-sklearn '
    '--o-classification "{0}:' + str(c) + '/rep_seqs_tax_assignments.qza" '
    '--i-classifier "{0}/classifier.qza" '
    '--i-reads {1} --p-confidence ' + str(c) + '; '
    'qiime tools export "{0}:' + str(c) + '/rep_seqs_tax_assignments.qza" --output-dir "{0}:' + str(c) + '"; '
    'mv "{0}:' + str(c) + '/taxonomy.tsv" "{0}:' + str(c) + '/query_tax_assignments.txt"; 'for c in confidences)

command_template += 'rm "{0}/classifier.qza"; rmdir "{0}"'

(dataset_reference_combinations, reference_dbs) = recall_novel_taxa_dirs(\
    data_dir, databases, iterations, ref_seqs='ref_seqs.qza',
    ref_taxa='ref_taxa.qza', max_level=6, min_level=5, multilevel=False)

commands = parameter_sweep(data_dir, results_dir, reference_dbs,
                           dataset_reference_combinations,
                           method_parameters_combinations, command_template,
                           infile='query.qza', output_name='rep_seqs_tax_assignments.qza')

In [16]:
print(len(commands), 'commands')
print('\n\n'.join(commands[0].split(';')))

80 commands
mkdir -p "../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/naive-bayes-bespoke/0.001::[6,6]"

 qiime feature-classifier fit-classifier-naive-bayes --o-classifier "../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/naive-bayes-bespoke/0.001::[6,6]/classifier.qza" --i-reference-reads ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/ref_seqs.qza --i-reference-taxonomy ../../../tax-credit-data-bk/data/cross-validated/S1-REF-iter0/ref_taxa.qza --p-classify--alpha 0.001 --p-classify--fit-prior  --p-feat-ext--ngram-range [6,6]

 mkdir -p "../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/naive-bayes-bespoke/0.001::[6,6]:0.5"

 qiime feature-classifier classify-sklearn --o-classification "../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/naive-bayes-bespoke/0.001::[6,6]:0.5/rep_seqs_tax_assignments.qza" --i-classifier "../../cross-validated-silva/S1-REF-iter0/S1-REF-iter0/naive-bayes-bespoke/0.001::[6,6]/classifier.qza" --i-reads ../../../tax-credit-dat

In [ ]:
Parallel(n_jobs=23)(delayed(system)(command) for command in commands);

### SEPP
In order to run SEPP, you need to install q2-fragment-insertion https://github.com/biocore/q2-fragment-insertion 

The wrapper script can be found here https://github.com/sjanssen2/q2-fragment-insertion/blob/tax-credit/tax-credit/wrapper_sepp.py

There are two independent algorithms to assign a taxonomic lineage to a sequence:
  1. **sepp-paths**: the tree is traversed from the inserted tip towards the root node and taxonomic labels along this path consitute the lineage. This highly depends on the quality of the taxonomic labels of the used reference phylogeny!
  2. **sepp-otus**: the closest OTU tip to the inserted sequences is located in the insertion tree and its the OTU's lineage is inherited for the sequence.
  
`reference_name` shall protocol which reference phylogeny/alignment was used for the SEPP run, but it does not influence anything in the wrapper. If you want use actually a different reference, maybe Silva, use parameters `reference_alignment` and `reference_phylogeny`. Method name and parameter `method` also coincide to satisfy the tax-credit framework. The called binary is the same, but we don't want to "merge" results of those two fundamentally different algorithms into the same "method" bin of tax-credit. But "method" here is more than a parameter.

Parameter `--cross_validate` ensures that SEPP first removes all sequences with IDs found in the input from reference phylogeny/alignment prior to inserting the query sequences into the (now reduced) reference and thus satisfying the nature of a cross validation.

In [25]:
# this filepath needs to point to the directory in which the SEPP wrapper can be found
fp_wrapper_script = '../../../q2-fragment-insertion/tax-credit/'

method_parameters_combinations = {
    'sepp-paths': {'method': ['path'], 'reference_name': ['gg138']},
    'sepp-otus': {'method': ['otus'], 'reference_name': ['gg138']}
}

command_template = "%s/wrapper_sepp.py {1} {0}/query_tax_assignments.txt {5} --cross_validate" % (
    fp_wrapper_script)


commands = parameter_sweep(data_dir, results_dir, reference_dbs,
                           dataset_reference_combinations,
                           method_parameters_combinations, command_template,
                           infile='query.qza', output_name='rep_seqs_tax_assignments.qza')

In [26]:
print(commands[0])

../../../q2-fragment-insertion/tax-credit//wrapper_sepp.py ../../../tax-credit-data/data/cross-validated/FL-REF-iter0/query.qza ../../data/precomputed-results/cross-validated/FL-REF-iter0/FL-REF-iter0/sepp-paths/path:gg138/query_tax_assignments.txt --reference_name gg138 --method path --cross_validate


## Move result files to repository

Add results to the tax-credit directory (e.g., to push these results to the repository or compare with other precomputed results in downstream analysis steps). The precomputed_results_dir path and methods_dirs glob below should not need to be changed unless if substantial changes were made to filepaths in the preceding cells.

In [18]:
precomputed_results_dir = join(project_dir, "data", "precomputed-results", analysis_name)
method_dirs = glob(join(results_dir, '*', '*', '*', '*'))
move_results_to_repository(method_dirs, precomputed_results_dir)